In [1]:
import pandas as pd
import os
import numpy as np
import requests
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data
from get_electricity_prices import get_data, get_todays_data
from get_weather_data import get_historical_weather, get_weather_forecast



In [2]:
load_dotenv()
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")

2024-12-28 21:14:56,380 INFO: Initializing external client
2024-12-28 21:14:56,381 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-28 21:14:57,916 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [3]:
forecast = get_weather_forecast("Luleå", "X", "X", 65.5841, 22.1547)
print(forecast)

Coordinates 65.58380889892578°N 22.155181884765625°E
Elevation 15.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
              time_start  temperature_2m  precipitation  snow_depth  \
0    2024-12-28T00:00:00        2.474000            0.0        0.13   
1    2024-12-28T01:00:00        2.724000            0.0        0.13   
2    2024-12-28T02:00:00        2.924000            0.0        0.13   
3    2024-12-28T03:00:00        3.024000            0.0        0.13   
4    2024-12-28T04:00:00        2.724000            0.0        0.13   
..                   ...             ...            ...         ...   
163  2025-01-03T19:00:00      -15.500000            0.0        0.14   
164  2025-01-03T20:00:00      -15.950000            0.0        0.14   
165  2025-01-03T21:00:00      -16.350000            0.0        0.14   
166  2025-01-03T22:00:00      -16.799999            0.0        0.14   
167  2025-01-03T23:00:00      -17.250000            0.0        0.14   

     pressure_msl  cl

In [4]:
price_fg = fs.get_feature_group(
    name='electricity_price_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)

In [5]:
weather_fg.insert(forecast)

2024-12-28 21:15:11,218 INFO: 	5 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393145


Uploading Dataframe: 100.00% |██████████| Rows 168/168 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_1_offline_fg_materialization/executions


(Job('weather_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation",
           "min_value": -0.0,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695303
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 168,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-28T08:15:11.000217Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_col

In [6]:
price = get_todays_data("SE4")
price['time_start'] = price['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
print(price)
print(price.dtypes)

         date           time_start    price
0  2024-12-28  2024-12-28T00:00:00  0.32504
1  2024-12-28  2024-12-28T01:00:00  0.28800
2  2024-12-28  2024-12-28T02:00:00  0.25451
3  2024-12-28  2024-12-28T03:00:00  0.23971
4  2024-12-28  2024-12-28T04:00:00  0.24155
5  2024-12-28  2024-12-28T05:00:00  0.23753
6  2024-12-28  2024-12-28T06:00:00  0.25049
7  2024-12-28  2024-12-28T07:00:00  0.28043
8  2024-12-28  2024-12-28T08:00:00  0.32321
9  2024-12-28  2024-12-28T09:00:00  0.34936
10 2024-12-28  2024-12-28T10:00:00  0.34626
11 2024-12-28  2024-12-28T11:00:00  0.35074
12 2024-12-28  2024-12-28T12:00:00  0.34970
13 2024-12-28  2024-12-28T13:00:00  0.35188
14 2024-12-28  2024-12-28T14:00:00  0.34202
15 2024-12-28  2024-12-28T15:00:00  0.34443
16 2024-12-28  2024-12-28T16:00:00  0.35028
17 2024-12-28  2024-12-28T17:00:00  0.33548
18 2024-12-28  2024-12-28T18:00:00  0.33938
19 2024-12-28  2024-12-28T19:00:00  0.29132
20 2024-12-28  2024-12-28T20:00:00  0.05964
21 2024-12-28  2024-12-28T21:00:

In [7]:
price_fg.insert(price)

2024-12-28 21:15:28,845 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393146


Uploading Dataframe: 100.00% |██████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: electricity_price_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/electricity_price_data_1_offline_fg_materialization/executions


(Job('electricity_price_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "price",
           "min_value": -0.1,
           "max_value": 5000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695305
         }
       },
       "result": {
         "observed_value": 0.03166,
         "element_count": 24,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-12-28T08:15:28.000845Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "succ